In [17]:
import numpy as np 
import pandas as pd
import math

In [18]:
train_df = pd.read_csv('data/trainset.csv')
test_df = pd.read_csv('data/testset.csv')
cv_df = pd.read_csv('data/cvset.csv')

In [27]:
train_df = pd.read_csv('data/trainset_pot_inc.csv')
test_df = pd.read_csv('data/testset_pot_inc.csv')
cv_df = pd.read_csv('data/cvset_pot_inc.csv')

In [28]:
x_train = train_df[train_df.columns[:-1]]
y_train = train_df[train_df.columns[-1:]]
x_test = test_df[test_df.columns[:-1]]
y_test = test_df[test_df.columns[-1:]]
x_cv = cv_df[cv_df.columns[:-1]]
y_cv = cv_df[cv_df.columns[-1:]]

In [29]:
type(y_train), type(y_test), type(y_cv)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [30]:
def estimateGaussian(X):
    mean = X.mean()

    m, n = X.shape

    sigma = np.zeros((n,n))

    #sigma (sigma^2) = each feature's variance
    for i in range(0,m):
        sigma = sigma + (X.iloc[i] - mean).values.reshape(n,1).dot((X.iloc[i] - mean).values.reshape(1, n))

    sigma = sigma / m

    return mean, sigma


In [31]:
def multivariateGaussian(X, mean, sigma):
    m, n = X.shape
    X = X.values - mean.values.reshape(1,n)
    
    #p-value for all entries in X, vectorized. can be replaced by multivariate_normal from scipy.stats
    p = (1 / (math.pow((2 * math.pi), n / 2) * math.pow(np.linalg.det(sigma),0.5))) * np.exp(-0.5 * np.sum(X.dot(np.linalg.pinv(sigma)) * X, axis=1))

    return p

In [32]:
def selectThreshold(yval, pval):
    yval = np.squeeze(yval.values).astype(int)

    max_epsilon = 0
    max_F1 = 0

    stepsize = (max(pval) - min(pval)) / 1000
    for epsilon in np.arange(min(pval), max(pval), stepsize):
        predictions = (pval < epsilon).astype(int)

        tp = np.sum((predictions == 1).astype(int) & (yval == 1).astype(int))
        fp = np.sum((predictions == 1).astype(int) & (yval == 0).astype(int))
        fn = np.sum((predictions == 0).astype(int) & (yval == 1).astype(int))

        prec = (tp) / (tp + fp)
        rec = (tp) / (tp + fn)

        F1 = (2 * prec * rec) / (prec + rec)

        if F1 > max_F1:
            max_F1 = F1
            max_epsilon = epsilon

    return max_epsilon, max_F1

In [33]:
mean, sigma = estimateGaussian(x_train)

pval_train = multivariateGaussian(x_train, mean, sigma)

pval_cv = multivariateGaussian(x_cv, mean, sigma)

epsilon, F1 = selectThreshold(y_cv, pval_cv)

print("Found max epsilon = " + str(epsilon) + ", max F1 = " + str(F1))

pval_test = multivariateGaussian(x_test, mean, sigma)

predictions = (pval_test < epsilon).astype(int)
y_test = np.squeeze(y_test.values).astype(int)

tp = np.sum((predictions == 1).astype(int) & (y_test == 1).astype(int))
fp = np.sum((predictions == 1).astype(int) & (y_test == 0).astype(int))
fn = np.sum((predictions == 0).astype(int) & (y_test == 1).astype(int))
tn = np.sum((predictions == 0).astype(int) & (y_test == 0).astype(int))

prec = (tp) / (tp + fp)
rec = (tp) / (tp + fn)

F1 = (2 * prec * rec) / (prec + rec)

print("Precision = " + str(prec) + ", Recall = " + str(rec), "F1 = " + str(F1))

ZeroDivisionError: float division by zero